In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import timedelta
Project_Path='D:/Minor proj data'

In [4]:
Scans_Data=pd.read_csv(Project_Path+ '/Scan_Data2.csv',low_memory=False,sep=',',index_col=0)
Scans_Data

,Date,Day Name,Day,Mouth,Year,Time,Latitude,Longitude,Zone,Slot_id,Legal,Covid,Lockdown
0,2018-01-03,Wednesday,3,1,2018,15:25,40.629133,22.953210,A,17622,1,0,0
1,2018-01-03,Wednesday,3,1,2018,15:06,40.629035,22.947317,A,17813,1,0,0
2,2018-01-03,Wednesday,3,1,2018,15:07,40.628861,22.947690,A,17813,0,0,0
3,2018-01-03,Wednesday,3,1,2018,15:07,40.629031,22.947298,A,17813,1,0,0
4,2018-01-03,Wednesday,3,1,2018,15:07,40.628900,22.947709,A,17813,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2018-01-04,Thursday,4,1,2018,12:21,40.629312,22.951746,A,17644,1,0,0
996,2018-01-04,Thursday,4,1,2018,12:48,40.636055,22.945928,A,20548,1,0,0
997,2018-01-04,Thursday,4,1,2018,15:24,40.626885,22.952857,A,17182,0,0,0
998,2018-01-04,Thursday,4,1,2018,12:48,40.636424,22.946217,A,20548,1,0,0


In [12]:
Date_List=[]
Scans_List=Scans_Data.values.tolist()
for i in tqdm(range(0,len(Scans_List))):
    Date = Scans_List[i][0]
    y,m,d = Date.split('-')
    D=m + '/' + d + '/' +y
    Date_List.append(D)
Scans_Data=Scans_Data.drop(['Date'], axis=1)
Scans_Data.insert(1, "Date", Date_List, True)
Scans_Data

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]


ValueError: not enough values to unpack (expected 3, got 1)

In [13]:
#I convert 'Date' and 'Time' columns to pandas datetime to convert time to time-int (Seconds)
Scans_Data['Date_Time'] = Scans_Data['Date'].map(str)+ ' ' + Scans_Data['Time'].map(str)
Scans_Data['Date_Time'] = pd.to_datetime(Scans_Data['Date_Time'])
Scans_Data['Time_Int'] = (Scans_Data['Date_Time']-Scans_Data['Date_Time'].dt.normalize()).dt.total_seconds()
Scans_Data=Scans_Data.drop(['Time'], axis=1)
Scans_Data=Scans_Data.drop(['Date_Time'], axis=1)
Scans_Data

,Day Name,Date,Day,Mouth,Year,Latitude,Longitude,Zone,Slot_id,Legal,Covid,Lockdown,Time_Int
0,Wednesday,01/03/2018,3,1,2018,40.629133,22.953210,A,17622,1,0,0,55500.0
1,Wednesday,01/03/2018,3,1,2018,40.629035,22.947317,A,17813,1,0,0,54360.0
2,Wednesday,01/03/2018,3,1,2018,40.628861,22.947690,A,17813,0,0,0,54420.0
3,Wednesday,01/03/2018,3,1,2018,40.629031,22.947298,A,17813,1,0,0,54420.0
4,Wednesday,01/03/2018,3,1,2018,40.628900,22.947709,A,17813,1,0,0,54420.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Thursday,01/04/2018,4,1,2018,40.629312,22.951746,A,17644,1,0,0,44460.0
996,Thursday,01/04/2018,4,1,2018,40.636055,22.945928,A,20548,1,0,0,46080.0
997,Thursday,01/04/2018,4,1,2018,40.626885,22.952857,A,17182,0,0,0,55440.0
998,Thursday,01/04/2018,4,1,2018,40.636424,22.946217,A,20548,1,0,0,46080.0


In [14]:
#Knowing from the previous notebook what are the times when I have recorded scans (6:00, 7:00,..., 19:00, 20:00) 
#I give their imprint in seconds, for example (6:00 = 21600) and then I capture her with their center. eg (6:00 -> 6:30) 
#and therefore (21600->23400). So I ended up with a list of 15 hours that will be used to sort the scans into time slots
Time_Slots=[21600,25200,28800,32400,36000,39600,43200,46800,50400,54000,57600,61200,64800,68400,72000]
Time_SlotsCenter=[]
for i in range (0,len(Time_Slots)):
    Time_SlotsCenter.append(Time_Slots[i]+1800)
Time_Slots=Time_SlotsCenter
Time_Slots

[23400,
 27000,
 30600,
 34200,
 37800,
 41400,
 45000,
 48600,
 52200,
 55800,
 59400,
 63000,
 66600,
 70200,
 73800]

In [15]:
Scan_List=Scans_Data.values.tolist()
Real_Time_Slots=[]
Time_Distance=[]
for i in tqdm(range(0,len(Scan_List))):
    Real_Time=Scan_List[i][12]
    Distances=[]
    Slots=[]
    
    for j in range (0,len(Time_Slots)):
        Distances.append(abs(Time_Slots[j]-Real_Time))
        
    Slots=np.column_stack((Time_Slots, Distances))
    Slots = sorted(Slots, key=lambda x: x[1])
    Real_Time_Slots.append(Slots[0][0])
    Time_Distance.append(Slots[0][1])
Scans_Data.insert(13, "Time_Slot", Real_Time_Slots, True)   
Scans_Data.insert(14, "Time_Distance", Time_Distance, True)   
Scans_Data

100%|███████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 24677.60it/s]


,Day Name,Date,Day,Mouth,Year,Latitude,Longitude,Zone,Slot_id,Legal,Covid,Lockdown,Time_Int,Time_Slot,Time_Distance
0,Wednesday,01/03/2018,3,1,2018,40.629133,22.953210,A,17622,1,0,0,55500.0,55800.0,300.0
1,Wednesday,01/03/2018,3,1,2018,40.629035,22.947317,A,17813,1,0,0,54360.0,55800.0,1440.0
2,Wednesday,01/03/2018,3,1,2018,40.628861,22.947690,A,17813,0,0,0,54420.0,55800.0,1380.0
3,Wednesday,01/03/2018,3,1,2018,40.629031,22.947298,A,17813,1,0,0,54420.0,55800.0,1380.0
4,Wednesday,01/03/2018,3,1,2018,40.628900,22.947709,A,17813,1,0,0,54420.0,55800.0,1380.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Thursday,01/04/2018,4,1,2018,40.629312,22.951746,A,17644,1,0,0,44460.0,45000.0,540.0
996,Thursday,01/04/2018,4,1,2018,40.636055,22.945928,A,20548,1,0,0,46080.0,45000.0,1080.0
997,Thursday,01/04/2018,4,1,2018,40.626885,22.952857,A,17182,0,0,0,55440.0,55800.0,360.0
998,Thursday,01/04/2018,4,1,2018,40.636424,22.946217,A,20548,1,0,0,46080.0,45000.0,1080.0


In [16]:
Scans_Data.drop(Scans_Data.index[Scans_Data['Lockdown'] == 1], inplace=True)
Scans_Data=Scans_Data.reset_index()
del Scans_Data["index"]
Scans_Data

,Day Name,Date,Day,Mouth,Year,Latitude,Longitude,Zone,Slot_id,Legal,Covid,Lockdown,Time_Int,Time_Slot,Time_Distance
0,Wednesday,01/03/2018,3,1,2018,40.629133,22.953210,A,17622,1,0,0,55500.0,55800.0,300.0
1,Wednesday,01/03/2018,3,1,2018,40.629035,22.947317,A,17813,1,0,0,54360.0,55800.0,1440.0
2,Wednesday,01/03/2018,3,1,2018,40.628861,22.947690,A,17813,0,0,0,54420.0,55800.0,1380.0
3,Wednesday,01/03/2018,3,1,2018,40.629031,22.947298,A,17813,1,0,0,54420.0,55800.0,1380.0
4,Wednesday,01/03/2018,3,1,2018,40.628900,22.947709,A,17813,1,0,0,54420.0,55800.0,1380.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Thursday,01/04/2018,4,1,2018,40.629312,22.951746,A,17644,1,0,0,44460.0,45000.0,540.0
996,Thursday,01/04/2018,4,1,2018,40.636055,22.945928,A,20548,1,0,0,46080.0,45000.0,1080.0
997,Thursday,01/04/2018,4,1,2018,40.626885,22.952857,A,17182,0,0,0,55440.0,55800.0,360.0
998,Thursday,01/04/2018,4,1,2018,40.636424,22.946217,A,20548,1,0,0,46080.0,45000.0,1080.0


In [17]:
Scans_Data2=Scans_Data['Slot_id'],Scans_Data['Date'],Scans_Data['Day Name'], Scans_Data['Time_Int'], Scans_Data['Time_Slot'],Scans_Data['Covid'],Scans_Data['Legal']
Headers=['Slot_id','Date','Day Name','Time_Int','Time_Slot','Covid','Legal']
Scans_Data2 = pd.concat(Scans_Data2, axis=1, keys=Headers)
Scans_Data2

,Slot_id,Date,Day Name,Time_Int,Time_Slot,Covid,Legal
0,17622,01/03/2018,Wednesday,55500.0,55800.0,0,1
1,17813,01/03/2018,Wednesday,54360.0,55800.0,0,1
2,17813,01/03/2018,Wednesday,54420.0,55800.0,0,0
3,17813,01/03/2018,Wednesday,54420.0,55800.0,0,1
4,17813,01/03/2018,Wednesday,54420.0,55800.0,0,1
...,...,...,...,...,...,...,...
995,17644,01/04/2018,Thursday,44460.0,45000.0,0,1
996,20548,01/04/2018,Thursday,46080.0,45000.0,0,1
997,17182,01/04/2018,Thursday,55440.0,55800.0,0,0
998,20548,01/04/2018,Thursday,46080.0,45000.0,0,1


In [18]:
Scans_Data2['Legal'] = Scans_Data2['Legal'].replace(1, 'Legal')
Scans_Data2['Legal'] = Scans_Data2['Legal'].replace(0, 'Ilegal')
Scans_Data2

,Slot_id,Date,Day Name,Time_Int,Time_Slot,Covid,Legal
0,17622,01/03/2018,Wednesday,55500.0,55800.0,0,Legal
1,17813,01/03/2018,Wednesday,54360.0,55800.0,0,Legal
2,17813,01/03/2018,Wednesday,54420.0,55800.0,0,Ilegal
3,17813,01/03/2018,Wednesday,54420.0,55800.0,0,Legal
4,17813,01/03/2018,Wednesday,54420.0,55800.0,0,Legal
...,...,...,...,...,...,...,...
995,17644,01/04/2018,Thursday,44460.0,45000.0,0,Legal
996,20548,01/04/2018,Thursday,46080.0,45000.0,0,Legal
997,17182,01/04/2018,Thursday,55440.0,55800.0,0,Ilegal
998,20548,01/04/2018,Thursday,46080.0,45000.0,0,Legal


In [24]:
# creating initial dataframe
#Legality_Types = ('Legal','Ilegal')
#Legaldf = pd.DataFrame(Scans_Data2, columns=['Legal'])
# generate binary values using get_dummies
#Legality = pd.get_dummies(Legaldf, columns=["Legal"], prefix=["Legality"] )
# merge with main df bridge_df on key values
#Scans_Data2 = Scans_Data2.join(Legality)
#Scans_Data2=Scans_Data2.drop(['Legal'], axis=1)
Scans_Data2.to_csv(Project_Path+ '/Scan_Data_Reg_2.1.csv')
Scans_Data2

,Slot_id,Date,Day Name,Time_Int,Time_Slot,Covid,Legality_Ilegal,Legality_Legal
0,17622,01/03/2018,Wednesday,55500.0,55800.0,0,False,True
1,17813,01/03/2018,Wednesday,54360.0,55800.0,0,False,True
2,17813,01/03/2018,Wednesday,54420.0,55800.0,0,True,False
3,17813,01/03/2018,Wednesday,54420.0,55800.0,0,False,True
4,17813,01/03/2018,Wednesday,54420.0,55800.0,0,False,True
...,...,...,...,...,...,...,...,...
995,17644,01/04/2018,Thursday,44460.0,45000.0,0,False,True
996,20548,01/04/2018,Thursday,46080.0,45000.0,0,False,True
997,17182,01/04/2018,Thursday,55440.0,55800.0,0,True,False
998,20548,01/04/2018,Thursday,46080.0,45000.0,0,False,True


In [25]:
Scans_Data2=pd.read_csv(Project_Path+ '/Scan_Data_Reg_2.1.csv',low_memory=False,sep=',',index_col=0)
Scans_Data2['Key'] = Scans_Data2['Slot_id'].map(str)+ '+' +Scans_Data2['Date'].map(str) + '+' + Scans_Data2['Day Name'].map(str)+ '+' + Scans_Data2['Time_Slot'].map(str)+ '+'+Scans_Data2['Covid'].map(str)
Scans_Data2=Scans_Data2['Key'],Scans_Data2['Time_Int'],Scans_Data2['Legality_Ilegal'],Scans_Data2['Legality_Legal']
Headers=['Key','Time_Int','Legality_Ilegal','Legality_Legal']
Scans_Data2 = pd.concat(Scans_Data2, axis=1, keys=Headers)
Scans_Data2

,Key,Time_Int,Legality_Ilegal,Legality_Legal
0,17622+01/03/2018+Wednesday+55800.0+0,55500.0,False,True
1,17813+01/03/2018+Wednesday+55800.0+0,54360.0,False,True
2,17813+01/03/2018+Wednesday+55800.0+0,54420.0,True,False
3,17813+01/03/2018+Wednesday+55800.0+0,54420.0,False,True
4,17813+01/03/2018+Wednesday+55800.0+0,54420.0,False,True
...,...,...,...,...
995,17644+01/04/2018+Thursday+45000.0+0,44460.0,False,True
996,20548+01/04/2018+Thursday+45000.0+0,46080.0,False,True
997,17182+01/04/2018+Thursday+55800.0+0,55440.0,True,False
998,20548+01/04/2018+Thursday+45000.0+0,46080.0,False,True


In [26]:
Time_Int = Scans_Data2.groupby(["Key"]).Time_Int.mean().reset_index()
ILegal_Sum = Scans_Data2.groupby(["Key"]).Legality_Ilegal.sum().reset_index()
Legal_Sum= Scans_Data2.groupby(["Key"]).Legality_Legal.sum().reset_index()
Legal_Sum=Legal_Sum['Key'],Time_Int['Time_Int'],ILegal_Sum['Legality_Ilegal'],Legal_Sum['Legality_Legal']
Headers=['Key','Time_Int','Legality_Ilegal','Legality_Legal']
Legal_illegal = pd.concat(Legal_Sum, axis=1, keys=Headers)
Legal_illegal

,Key,Time_Int,Legality_Ilegal,Legality_Legal
0,10+01/03/2018+Wednesday+55800.0+0,55873.333333,7,2
1,11+01/03/2018+Wednesday+55800.0+0,56415.000000,7,5
2,17182+01/03/2018+Wednesday+30600.0+0,29160.000000,12,0
3,17182+01/03/2018+Wednesday+37800.0+0,39600.000000,1,0
4,17182+01/03/2018+Wednesday+41400.0+0,40016.250000,14,2
...,...,...,...,...
110,79+01/03/2018+Wednesday+55800.0+0,57600.000000,1,0
111,87+01/03/2018+Wednesday+30600.0+0,29820.000000,1,0
112,87+01/03/2018+Wednesday+41400.0+0,39720.000000,0,1
113,87+01/04/2018+Thursday+41400.0+0,42600.000000,1,0


In [27]:
Legal_illegal['Sum']=Legal_illegal['Legality_Ilegal']+Legal_illegal['Legality_Legal']
Legal_illegal.drop(Legal_illegal[Legal_illegal.Sum <= 2].index, inplace=True)
Legal_illegal=Legal_illegal.reset_index()
del Legal_illegal["index"]
Legal_illegal

,Key,Time_Int,Legality_Ilegal,Legality_Legal,Sum
0,10+01/03/2018+Wednesday+55800.0+0,55873.333333,7,2,9
1,11+01/03/2018+Wednesday+55800.0+0,56415.000000,7,5,12
2,17182+01/03/2018+Wednesday+30600.0+0,29160.000000,12,0,12
3,17182+01/03/2018+Wednesday+41400.0+0,40016.250000,14,2,16
4,17182+01/03/2018+Wednesday+59400.0+0,58903.125000,25,7,32
...,...,...,...,...,...
74,28323+01/03/2018+Wednesday+30600.0+0,32085.000000,4,4,8
75,28323+01/03/2018+Wednesday+41400.0+0,41635.714286,7,7,14
76,28323+01/03/2018+Wednesday+55800.0+0,54570.000000,4,2,6
77,28323+01/04/2018+Thursday+45000.0+0,43620.000000,5,1,6


In [28]:
Scan_List2=Legal_illegal.values.tolist()
Ilegality_Rate=[]
for i in range (0,len(Scan_List2)):
    if Scan_List2[i][2]==0:
        Ilegality_Rate.append(0)
    else:
        Ilegality_Rate.append(Scan_List2[i][2]/(Scan_List2[i][2]+Scan_List2[i][3]))
Legal_illegal.insert(3, "Ilegality_Rate", Ilegality_Rate, True)
Legal_illegal=Legal_illegal.drop(['Legality_Ilegal'], axis=1)
Legal_illegal=Legal_illegal.drop(['Legality_Legal'], axis=1)
Legal_illegal

,Key,Time_Int,Ilegality_Rate,Sum
0,10+01/03/2018+Wednesday+55800.0+0,55873.333333,0.777778,9
1,11+01/03/2018+Wednesday+55800.0+0,56415.000000,0.583333,12
2,17182+01/03/2018+Wednesday+30600.0+0,29160.000000,1.000000,12
3,17182+01/03/2018+Wednesday+41400.0+0,40016.250000,0.875000,16
4,17182+01/03/2018+Wednesday+59400.0+0,58903.125000,0.781250,32
...,...,...,...,...
74,28323+01/03/2018+Wednesday+30600.0+0,32085.000000,0.500000,8
75,28323+01/03/2018+Wednesday+41400.0+0,41635.714286,0.500000,14
76,28323+01/03/2018+Wednesday+55800.0+0,54570.000000,0.666667,6
77,28323+01/04/2018+Thursday+45000.0+0,43620.000000,0.833333,6


In [29]:
Scan_List2=Legal_illegal.values.tolist()
Slot_id=[]
Date=[]
Day_Name=[]
Slot_Timeint=[]
Covid=[]
for i in tqdm(range(0,len(Scan_List2))):
    Slot_id_Value,Date_Value,Day_Name_Value,Slot_Timeint_Value,Covid_Value=Scan_List2[i][0].split('+')
    Slot_id.append(Slot_id_Value)
    Date.append(Date_Value)
    Day_Name.append(Day_Name_Value)
    Slot_Timeint.append(Slot_Timeint_Value)
    Covid.append(Covid_Value)
Legal_illegal.insert(1, "Slot_id", Slot_id, True)
Legal_illegal.insert(2, "Date", Date, True)
Legal_illegal.insert(3, "Day_Name",  Day_Name, True)
Legal_illegal.insert(4, "Slot_Timeint", Slot_Timeint, True)
Legal_illegal.insert(5, "Covid", Covid, True)
Legal_illegal=Legal_illegal.drop(['Key'], axis=1)
Legal_illegal

100%|██████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<?, ?it/s]


,Slot_id,Date,Day_Name,Slot_Timeint,Covid,Time_Int,Ilegality_Rate,Sum
0,10,01/03/2018,Wednesday,55800.0,0,55873.333333,0.777778,9
1,11,01/03/2018,Wednesday,55800.0,0,56415.000000,0.583333,12
2,17182,01/03/2018,Wednesday,30600.0,0,29160.000000,1.000000,12
3,17182,01/03/2018,Wednesday,41400.0,0,40016.250000,0.875000,16
4,17182,01/03/2018,Wednesday,59400.0,0,58903.125000,0.781250,32
...,...,...,...,...,...,...,...,...
74,28323,01/03/2018,Wednesday,30600.0,0,32085.000000,0.500000,8
75,28323,01/03/2018,Wednesday,41400.0,0,41635.714286,0.500000,14
76,28323,01/03/2018,Wednesday,55800.0,0,54570.000000,0.666667,6
77,28323,01/04/2018,Thursday,45000.0,0,43620.000000,0.833333,6


In [30]:
Legal_illegal['Slot_Timeint'] = Legal_illegal['Slot_Timeint'].astype(float)
Legal_illegal['Time_Int'] = Legal_illegal['Time_Int'].astype(float)
Legal_illegal['Slot_Timeint'] = Legal_illegal['Slot_Timeint'].astype(int)
Legal_illegal['Time_Int'] = Legal_illegal['Time_Int'].astype(int)
Legal_illegal

,Slot_id,Date,Day_Name,Slot_Timeint,Covid,Time_Int,Ilegality_Rate,Sum
0,10,01/03/2018,Wednesday,55800,0,55873,0.777778,9
1,11,01/03/2018,Wednesday,55800,0,56415,0.583333,12
2,17182,01/03/2018,Wednesday,30600,0,29160,1.000000,12
3,17182,01/03/2018,Wednesday,41400,0,40016,0.875000,16
4,17182,01/03/2018,Wednesday,59400,0,58903,0.781250,32
...,...,...,...,...,...,...,...,...
74,28323,01/03/2018,Wednesday,30600,0,32085,0.500000,8
75,28323,01/03/2018,Wednesday,41400,0,41635,0.500000,14
76,28323,01/03/2018,Wednesday,55800,0,54570,0.666667,6
77,28323,01/04/2018,Thursday,45000,0,43620,0.833333,6


In [32]:
Column_Names=['Slot_id','Adress','Mean','Start','End','Capacity','Full_adress','Latitude','Longitude']

Parking_Slots2=pd.read_csv(Project_Path+ '/Parkink_Slot_Proccesed.csv',sep=',', names=Column_Names)

Parking_Slots2=Parking_Slots2.values.tolist()
Scans_List2=Legal_illegal.values.tolist()
Capacity=[]
for i in tqdm(range(0,len(Scans_List2))):
    for j in range (0,len(Parking_Slots2)):
        if int(Parking_Slots2[j][0])==int(Scans_List2[i][0]):
            c=int(Parking_Slots2[j][5])
            Capacity.append(c) 



Legal_illegal.insert(7, "Capacity",Capacity, True)
Legal_illegal


100%|████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 5785.95it/s]


,Slot_id,Date,Day_Name,Slot_Timeint,Covid,Time_Int,Ilegality_Rate,Capacity,Sum
0,10,01/03/2018,Wednesday,55800,0,55873,0.777778,6,9
1,11,01/03/2018,Wednesday,55800,0,56415,0.583333,18,12
2,17182,01/03/2018,Wednesday,30600,0,29160,1.000000,18,12
3,17182,01/03/2018,Wednesday,41400,0,40016,0.875000,18,16
4,17182,01/03/2018,Wednesday,59400,0,58903,0.781250,18,32
...,...,...,...,...,...,...,...,...,...
74,28323,01/03/2018,Wednesday,30600,0,32085,0.500000,9,8
75,28323,01/03/2018,Wednesday,41400,0,41635,0.500000,9,14
76,28323,01/03/2018,Wednesday,55800,0,54570,0.666667,9,6
77,28323,01/04/2018,Thursday,45000,0,43620,0.833333,9,6


In [33]:
Scan_List2=Legal_illegal.values.tolist()
Attention=[]
for i in range (0,len(Scan_List2)):
    if Scan_List2[i][8]>Scan_List2[i][7]:
        Attention.append(1)
    else:
        Attention.append(Scan_List2[i][8]/(Scan_List2[i][7]))
Legal_illegal.insert(7, "Attention", Attention, True)

Legal_illegal

,Slot_id,Date,Day_Name,Slot_Timeint,Covid,Time_Int,Ilegality_Rate,Attention,Capacity,Sum
0,10,01/03/2018,Wednesday,55800,0,55873,0.777778,1.000000,6,9
1,11,01/03/2018,Wednesday,55800,0,56415,0.583333,0.666667,18,12
2,17182,01/03/2018,Wednesday,30600,0,29160,1.000000,0.666667,18,12
3,17182,01/03/2018,Wednesday,41400,0,40016,0.875000,0.888889,18,16
4,17182,01/03/2018,Wednesday,59400,0,58903,0.781250,1.000000,18,32
...,...,...,...,...,...,...,...,...,...,...
74,28323,01/03/2018,Wednesday,30600,0,32085,0.500000,0.888889,9,8
75,28323,01/03/2018,Wednesday,41400,0,41635,0.500000,1.000000,9,14
76,28323,01/03/2018,Wednesday,55800,0,54570,0.666667,0.666667,9,6
77,28323,01/04/2018,Thursday,45000,0,43620,0.833333,0.666667,9,6


In [34]:
Legal_illegal.drop(Legal_illegal[Legal_illegal.Attention <= 0.2].index, inplace=True)
Legal_illegal.to_csv(Project_Path+ '/Scan_Data_For_Analysis.csv')
Legal_illegal=Legal_illegal.drop(['Attention'], axis=1)
Legal_illegal=Legal_illegal.reset_index()
del Legal_illegal["index"]
Legal_illegal

,Slot_id,Date,Day_Name,Slot_Timeint,Covid,Time_Int,Ilegality_Rate,Capacity,Sum
0,10,01/03/2018,Wednesday,55800,0,55873,0.777778,6,9
1,11,01/03/2018,Wednesday,55800,0,56415,0.583333,18,12
2,17182,01/03/2018,Wednesday,30600,0,29160,1.000000,18,12
3,17182,01/03/2018,Wednesday,41400,0,40016,0.875000,18,16
4,17182,01/03/2018,Wednesday,59400,0,58903,0.781250,18,32
...,...,...,...,...,...,...,...,...,...
69,28323,01/03/2018,Wednesday,30600,0,32085,0.500000,9,8
70,28323,01/03/2018,Wednesday,41400,0,41635,0.500000,9,14
71,28323,01/03/2018,Wednesday,55800,0,54570,0.666667,9,6
72,28323,01/04/2018,Thursday,45000,0,43620,0.833333,9,6


In [36]:
Holidays= pd.read_csv(Project_Path+ '/Holidays.txt', header = None,names=['Holidays'])
Holidays_List=Holidays.values.tolist()
Scan_List2=Legal_illegal.values.tolist()
Holidays=[]
for i in tqdm(range(0,len(Scan_List2))):
    Boulean=False
    for j in range (0,len(Holidays_List)):
        if Scan_List2[i][1]==Holidays_List[j][0]:
            Boulean=True
    
    if Boulean:
        Holidays.append(1)
    else:
        Holidays.append(0)
        
Legal_illegal.insert(7, "Holidays", Holidays, True)
Legal_illegal

100%|██████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:00<?, ?it/s]


,Slot_id,Date,Day_Name,Slot_Timeint,Covid,Time_Int,Ilegality_Rate,Holidays,Capacity,Sum
0,10,01/03/2018,Wednesday,55800,0,55873,0.777778,0,6,9
1,11,01/03/2018,Wednesday,55800,0,56415,0.583333,0,18,12
2,17182,01/03/2018,Wednesday,30600,0,29160,1.000000,0,18,12
3,17182,01/03/2018,Wednesday,41400,0,40016,0.875000,0,18,16
4,17182,01/03/2018,Wednesday,59400,0,58903,0.781250,0,18,32
...,...,...,...,...,...,...,...,...,...,...
69,28323,01/03/2018,Wednesday,30600,0,32085,0.500000,0,9,8
70,28323,01/03/2018,Wednesday,41400,0,41635,0.500000,0,9,14
71,28323,01/03/2018,Wednesday,55800,0,54570,0.666667,0,9,6
72,28323,01/04/2018,Thursday,45000,0,43620,0.833333,0,9,6


In [37]:
Scan_List2=Legal_illegal.values.tolist()

Date=[]
Year=[]
Mounth=[]
Key_Date=[]

for i in tqdm(range(0,len(Scan_List2))):
    m,d,y=Scan_List2[i][1].split('/')
    Mounth.append(int(m))
    Date.append(int(d))
    Year.append(int(y))
    D=y + '-' + m + '-' +d
    Key_Date.append(D)

Legal_illegal.insert(1, "Key_Date", Key_Date, True)
Legal_illegal.insert(2, "Date_Of_Month", Date, True)
Legal_illegal.insert(3, "Year", Year, True)
Legal_illegal.insert(4, "Month", Mounth, True)
Legal_illegal=Legal_illegal.drop(['Date'], axis=1)
Legal_illegal

100%|██████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:00<?, ?it/s]


,Slot_id,Key_Date,Date_Of_Month,Year,Month,Day_Name,Slot_Timeint,Covid,Time_Int,Ilegality_Rate,Holidays,Capacity,Sum
0,10,2018-01-03,3,2018,1,Wednesday,55800,0,55873,0.777778,0,6,9
1,11,2018-01-03,3,2018,1,Wednesday,55800,0,56415,0.583333,0,18,12
2,17182,2018-01-03,3,2018,1,Wednesday,30600,0,29160,1.000000,0,18,12
3,17182,2018-01-03,3,2018,1,Wednesday,41400,0,40016,0.875000,0,18,16
4,17182,2018-01-03,3,2018,1,Wednesday,59400,0,58903,0.781250,0,18,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,28323,2018-01-03,3,2018,1,Wednesday,30600,0,32085,0.500000,0,9,8
70,28323,2018-01-03,3,2018,1,Wednesday,41400,0,41635,0.500000,0,9,14
71,28323,2018-01-03,3,2018,1,Wednesday,55800,0,54570,0.666667,0,9,6
72,28323,2018-01-04,4,2018,1,Thursday,45000,0,43620,0.833333,0,9,6


In [38]:
Legal_illegal.to_csv(Project_Path+ '/Scan_Data_Reg_2.2.csv')

In [39]:
Legal_illegal=pd.read_csv(Project_Path+ '/Scan_Data_Reg_2.2.csv',sep=',',index_col=0)
Legal_illegal

,Slot_id,Key_Date,Date_Of_Month,Year,Month,Day_Name,Slot_Timeint,Covid,Time_Int,Ilegality_Rate,Holidays,Capacity,Sum
0,10,2018-01-03,3,2018,1,Wednesday,55800,0,55873,0.777778,0,6,9
1,11,2018-01-03,3,2018,1,Wednesday,55800,0,56415,0.583333,0,18,12
2,17182,2018-01-03,3,2018,1,Wednesday,30600,0,29160,1.000000,0,18,12
3,17182,2018-01-03,3,2018,1,Wednesday,41400,0,40016,0.875000,0,18,16
4,17182,2018-01-03,3,2018,1,Wednesday,59400,0,58903,0.781250,0,18,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,28323,2018-01-03,3,2018,1,Wednesday,30600,0,32085,0.500000,0,9,8
70,28323,2018-01-03,3,2018,1,Wednesday,41400,0,41635,0.500000,0,9,14
71,28323,2018-01-03,3,2018,1,Wednesday,55800,0,54570,0.666667,0,9,6
72,28323,2018-01-04,4,2018,1,Thursday,45000,0,43620,0.833333,0,9,6


In [40]:
List=Legal_illegal.values.tolist()
Week_Day=[]
Legal_illegal=Legal_illegal.drop(['Day_Name'], axis=1)
Day_Name_To_Day_Slot=[['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],[1,2,3,4,5,6,7]]
for i in tqdm(range(0,len(List))):
    for j in range(0,7):
        if List[i][5]==Day_Name_To_Day_Slot[0][j]:
            Week_Day.append(Day_Name_To_Day_Slot[1][j])
   
Legal_illegal.insert(2, "Week_Day", Week_Day, True)
Legal_illegal

100%|██████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:00<?, ?it/s]


,Slot_id,Key_Date,Week_Day,Date_Of_Month,Year,Month,Slot_Timeint,Covid,Time_Int,Ilegality_Rate,Holidays,Capacity,Sum
0,10,2018-01-03,3,3,2018,1,55800,0,55873,0.777778,0,6,9
1,11,2018-01-03,3,3,2018,1,55800,0,56415,0.583333,0,18,12
2,17182,2018-01-03,3,3,2018,1,30600,0,29160,1.000000,0,18,12
3,17182,2018-01-03,3,3,2018,1,41400,0,40016,0.875000,0,18,16
4,17182,2018-01-03,3,3,2018,1,59400,0,58903,0.781250,0,18,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,28323,2018-01-03,3,3,2018,1,30600,0,32085,0.500000,0,9,8
70,28323,2018-01-03,3,3,2018,1,41400,0,41635,0.500000,0,9,14
71,28323,2018-01-03,3,3,2018,1,55800,0,54570,0.666667,0,9,6
72,28323,2018-01-04,4,4,2018,1,45000,0,43620,0.833333,0,9,6


In [41]:
def encode(data, col, max_val):
    data[col + '_Sin'] = np.cos(2 * np.pi * data[col]/max_val)
    return data
Legal_illegal = encode(Legal_illegal, 'Week_Day', 7)
Legal_illegal = encode(Legal_illegal, 'Month', 12)
Legal_illegal

,Slot_id,Key_Date,Week_Day,Date_Of_Month,Year,Month,Slot_Timeint,Covid,Time_Int,Ilegality_Rate,Holidays,Capacity,Sum,Week_Day_Sin,Month_Sin
0,10,2018-01-03,3,3,2018,1,55800,0,55873,0.777778,0,6,9,-0.900969,0.866025
1,11,2018-01-03,3,3,2018,1,55800,0,56415,0.583333,0,18,12,-0.900969,0.866025
2,17182,2018-01-03,3,3,2018,1,30600,0,29160,1.000000,0,18,12,-0.900969,0.866025
3,17182,2018-01-03,3,3,2018,1,41400,0,40016,0.875000,0,18,16,-0.900969,0.866025
4,17182,2018-01-03,3,3,2018,1,59400,0,58903,0.781250,0,18,32,-0.900969,0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,28323,2018-01-03,3,3,2018,1,30600,0,32085,0.500000,0,9,8,-0.900969,0.866025
70,28323,2018-01-03,3,3,2018,1,41400,0,41635,0.500000,0,9,14,-0.900969,0.866025
71,28323,2018-01-03,3,3,2018,1,55800,0,54570,0.666667,0,9,6,-0.900969,0.866025
72,28323,2018-01-04,4,4,2018,1,45000,0,43620,0.833333,0,9,6,-0.900969,0.866025


In [42]:
List=Legal_illegal.values.tolist()
Date_Sin=[]
Month_Sin_Rules=[[1,2,3,4,5,6,7,8,9,10,11,12],[31,29,31,30,31,30,31,31,30,31,30,31]]
for i in tqdm(range(0,len(List))):
    for j in range(0,12):
        if List[i][5]==Month_Sin_Rules[0][j]:
            Max=Month_Sin_Rules[1][j]
    sin=np.sin(2 * np.pi * List[i][2]/Max)
    Date_Sin.append(sin)
Legal_illegal.insert(2, "Date_Sin", Date_Sin, True)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:00<?, ?it/s]


In [43]:
Legal_illegal=Legal_illegal.drop(['Week_Day'], axis=1)
Legal_illegal=Legal_illegal.drop(['Month'], axis=1)
Legal_illegal=Legal_illegal.drop(['Date_Of_Month'], axis=1)
Legal_illegal=Legal_illegal.drop(['Sum'], axis=1)
Legal_illegal

,Slot_id,Key_Date,Date_Sin,Year,Slot_Timeint,Covid,Time_Int,Ilegality_Rate,Holidays,Capacity,Week_Day_Sin,Month_Sin
0,10,2018-01-03,0.571268,2018,55800,0,55873,0.777778,0,6,-0.900969,0.866025
1,11,2018-01-03,0.571268,2018,55800,0,56415,0.583333,0,18,-0.900969,0.866025
2,17182,2018-01-03,0.571268,2018,30600,0,29160,1.000000,0,18,-0.900969,0.866025
3,17182,2018-01-03,0.571268,2018,41400,0,40016,0.875000,0,18,-0.900969,0.866025
4,17182,2018-01-03,0.571268,2018,59400,0,58903,0.781250,0,18,-0.900969,0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...
69,28323,2018-01-03,0.571268,2018,30600,0,32085,0.500000,0,9,-0.900969,0.866025
70,28323,2018-01-03,0.571268,2018,41400,0,41635,0.500000,0,9,-0.900969,0.866025
71,28323,2018-01-03,0.571268,2018,55800,0,54570,0.666667,0,9,-0.900969,0.866025
72,28323,2018-01-04,0.724793,2018,45000,0,43620,0.833333,0,9,-0.900969,0.866025


In [44]:
Time=Legal_illegal['Slot_Timeint']/3600 #convert seconds to hours
Time=Time.astype(int) #i only get the hour and not the minutes
Time=Time.values.tolist()
NewT=[]
for i in range (0,len(Time)):
    Str=str(Time[i])
    if Time[i]>=10:
        NewT.append(Str)
    else:
        NewT.append('0'+Str)


Time=pd.DataFrame(NewT,columns=["Hour"])


Time= Time["Hour"].map(str)+ ':00' 
Key_Weather=Legal_illegal['Key_Date'].map(str) + ' ' + Time
Legal_illegal=Legal_illegal.drop(['Key_Date'], axis=1)
Legal_illegal.insert(1, "Key", Key_Weather, True)
Legal_illegal

,Slot_id,Key,Date_Sin,Year,Slot_Timeint,Covid,Time_Int,Ilegality_Rate,Holidays,Capacity,Week_Day_Sin,Month_Sin
0,10,2018-01-03 15:00,0.571268,2018,55800,0,55873,0.777778,0,6,-0.900969,0.866025
1,11,2018-01-03 15:00,0.571268,2018,55800,0,56415,0.583333,0,18,-0.900969,0.866025
2,17182,2018-01-03 08:00,0.571268,2018,30600,0,29160,1.000000,0,18,-0.900969,0.866025
3,17182,2018-01-03 11:00,0.571268,2018,41400,0,40016,0.875000,0,18,-0.900969,0.866025
4,17182,2018-01-03 16:00,0.571268,2018,59400,0,58903,0.781250,0,18,-0.900969,0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...
69,28323,2018-01-03 08:00,0.571268,2018,30600,0,32085,0.500000,0,9,-0.900969,0.866025
70,28323,2018-01-03 11:00,0.571268,2018,41400,0,41635,0.500000,0,9,-0.900969,0.866025
71,28323,2018-01-03 15:00,0.571268,2018,55800,0,54570,0.666667,0,9,-0.900969,0.866025
72,28323,2018-01-04 12:00,0.724793,2018,45000,0,43620,0.833333,0,9,-0.900969,0.866025


In [45]:
Legal_illegal['Slot_Timeint']=Legal_illegal['Slot_Timeint']/timedelta(days=1).total_seconds()
Legal_illegal['Time_Int']=Legal_illegal['Time_Int']/timedelta(days=1).total_seconds()
Legal_illegal=Legal_illegal.drop(['Year'],axis=1)

In [47]:
Legal_illegal.to_csv(Project_Path+ '/Scan_Data_Reg_2.3.csv')
Legal_illegal

,Slot_id,Key,Date_Sin,Slot_Timeint,Covid,Time_Int,Ilegality_Rate,Holidays,Capacity,Week_Day_Sin,Month_Sin
0,10,2018-01-03 15:00,0.571268,0.645833,0,0.646678,0.777778,0,6,-0.900969,0.866025
1,11,2018-01-03 15:00,0.571268,0.645833,0,0.652951,0.583333,0,18,-0.900969,0.866025
2,17182,2018-01-03 08:00,0.571268,0.354167,0,0.337500,1.000000,0,18,-0.900969,0.866025
3,17182,2018-01-03 11:00,0.571268,0.479167,0,0.463148,0.875000,0,18,-0.900969,0.866025
4,17182,2018-01-03 16:00,0.571268,0.687500,0,0.681748,0.781250,0,18,-0.900969,0.866025
...,...,...,...,...,...,...,...,...,...,...,...
69,28323,2018-01-03 08:00,0.571268,0.354167,0,0.371354,0.500000,0,9,-0.900969,0.866025
70,28323,2018-01-03 11:00,0.571268,0.479167,0,0.481887,0.500000,0,9,-0.900969,0.866025
71,28323,2018-01-03 15:00,0.571268,0.645833,0,0.631597,0.666667,0,9,-0.900969,0.866025
72,28323,2018-01-04 12:00,0.724793,0.520833,0,0.504861,0.833333,0,9,-0.900969,0.866025
